# Title

## How to Use:

0. **Run the first code cell** to load all imports and utilities (optimized for performance)
1. **Change the stock**: Edit `user_input = 'AAPL'` to any stock symbol
2. **Run the selection cell** to load the stock data
3. **Run the dashboard cell** to create the analysis

## Dashboard Components:

- **Main Chart**: Stock price with moving averages and Bollinger Bands
- **Volume**: Trading volume in millions (color-coded by price change)
- **RSI**: Relative Strength Index with overbought/oversold levels
- **Volatility**: Annualized volatility percentage
- **Cumulative Returns**: Total returns from start
- **MACD**: Moving Average Convergence Divergence with signal line and histogram
- **Max Drawdown**: Peak-to-trough decline percentage over time
- **Stochastic Oscillator**: %K and %D lines with overbought/oversold levels
- **Returns Distribution**: Histogram of daily returns
- **On-Balance Volume**: Cumulative volume indicator
- **Rolling Sharpe Ratio**: Risk-adjusted performance over 252-day windows

## Examples:
- `user_input = 'GOOGL'` for Google
- `user_input = 'TSLA'` for Tesla
- `user_input = 'NVDA'` for NVIDIA
- `user_input = 'MSFT'` for Microsoft

# Table of Contents

- [Setup and Configuration](#setup-and-configuration)
- [Stock Selection](#stock-selection)  
- [Technical Analysis Dashboard](#technical-analysis-dashboard)
- [Company Information](#company-information)
- [Analyst Price Targets](#analyst-price-targets)
- [Market Intelligence](#market-intelligence)

[Back to Top](#table-of-contents)

## Setup and Configuration

In [ ]:
# Core imports
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from statsmodels.tsa.stattools import coint
from statsmodels.tsa.vector_ar.vecm import coint_johansen
import scipy.stats as stats
from typing import Dict, List, Optional
from pprint import pprint
import warnings
warnings.filterwarnings('ignore')
import functions
import importlib
importlib.reload(functions)

# Additional imports for display and web requests
from IPython.display import Markdown, display
import requests
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter
import time
import json

# Display and utility imports
from IPython.display import HTML, display, Markdown
from io import BytesIO
import base64

# Import utility functions
from functions import (
    get_ticker, 
    fetch_stock_data, 
    setup_date_formatting, 
    humanize_number, 
    safe_float, 
    get_current_price_and_date,
    display_structured_data_as_markdown
)

In [ ]:
# Matplotlib Style Configuration
# Change this line to use different graph styles

plt.style.use('default')            # Clean white background

# Custom style settings (optional)
plt.rcParams.update({
    'figure.figsize': (12, 8),      # Default figure size
    'font.size': 10,                # Default font size
    'axes.labelsize': 12,           # Axis label size
    'axes.titlesize': 14,           # Title size
    'xtick.labelsize': 10,          # X-tick label size
    'ytick.labelsize': 10,          # Y-tick label size
    'legend.fontsize': 10,          # Legend font size
    'figure.dpi': 200,              # Resolution
})

# List available styles
#for i, style in enumerate(sorted(plt.style.available), 1):
    #print(f"  {i:2d}. {style}")

In [ ]:
# Utility function for making API requests to CNN Business
def make_cnn_api_request(url, max_retries=3, backoff_factor=1.0, timeout=10):
    """
    Make an API request to CNN Business with retry logic.
    
    Args:
        url: The API endpoint URL
        max_retries: Number of retry attempts
        backoff_factor: Factor for exponential backoff
        timeout: Request timeout in seconds
    
    Returns:
        Response object or None if all retries failed
    """
    session = requests.Session()
    retry = Retry(
        total=max_retries,
        backoff_factor=backoff_factor,
        status_forcelist=[500, 502, 503, 504]
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    
    try:
        response = session.get(url, timeout=timeout)
        return response
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return None

## Stock Selection
**How to change stocks:**
1. In the cell below, change `user_input = 'AAPL'` to any stock symbol you want
2. Examples: `'GOOGL'`, `'MSFT'`, `'TSLA'`, `'NVDA'`, `'AMZN'`, etc.
3. Run the cell above, then run the dashboard cell below

In [ ]:
selected_stock = 'AAPL'  # Change this to any stock symbol (e.g., 'AAPL', 'GOOGL', 'MSFT', etc.)
period = '1y'  # Change this to desired period (e.g., '1d', '1wk', '6mo', '1y', '2y', 'max' etc.) - x-axis labels will auto-adjust

In [ ]:
# STOCK SELECTION
# Change the stock symbol below to any stock you want to analyze
print(f"Fetching data for {selected_stock}...")

# Fetch stock data
data = fetch_stock_data(selected_stock, period)

if not data.empty:
    # Calculate technical indicators using utility function
    from functions import calculate_technical_indicators
    data = calculate_technical_indicators(data)
    
    # Calculate additional metrics
    data['Daily_Return'] = data['Close'].pct_change()
    data['Volatility'] = data['Daily_Return'].rolling(window=20).std() * np.sqrt(252)
    data['Cumulative_Return'] = (1 + data['Daily_Return']).cumprod() - 1
    
    ticker = get_ticker(selected_stock)  # Ensure ticker is set
    print(f"Successfully loaded {selected_stock}")
else:
    print(f"No data found for {selected_stock}")

## Technical Analysis Dashboard

In [ ]:
# Create dashboard visualization
print(f"Creating dashboard for {selected_stock}...")

# Period mapping for readable titles
period_names = {
    '1d': '1 Day', '5d': '5 Days', '1mo': '1 Month', '3mo': '3 Months',
    '6mo': '6 Months', '1y': '1 Year', '2y': '2 Years', '5y': '5 Years',
    '10y': '10 Years', 'ytd': 'Year to Date', 'max': 'Maximum'
}

period_display = period_names.get(period, period)
# Use the 'data' variable that was already loaded in the previous cell

# Pre-calculate all technical indicators (vectorized operations for speed)
close_price = data['Close']
rolling_std_20 = close_price.rolling(20).std()
daily_returns = data['Daily_Return']

# Bollinger Bands
upper_band = data['MA_20'] + (rolling_std_20 * 2)
lower_band = data['MA_20'] - (rolling_std_20 * 2)

# MACD (consolidated calculations)
ema_12, ema_26 = close_price.ewm(span=12).mean(), close_price.ewm(span=26).mean()
macd = ema_12 - ema_26
signal = macd.ewm(span=9).mean()
histogram = macd - signal

# Maximum Drawdown (optimized)
cumulative = (1 + daily_returns).cumprod()
drawdown = ((cumulative - cumulative.cummax()) / cumulative.cummax()) * 100

# Stochastic Oscillator
low_high_range = data['High'].rolling(14).max() - data['Low'].rolling(14).min()
k_percent = 100 * ((close_price - data['Low'].rolling(14).min()) / low_high_range)
d_percent = k_percent.rolling(3).mean()

# On-Balance Volume & Rolling Sharpe (vectorized)
obv = (np.sign(close_price.diff()) * data['Volume']).fillna(0).cumsum()
rolling_sharpe = (daily_returns.rolling(60).mean() / daily_returns.rolling(60).std()) * np.sqrt(252)

# Helper function to add value annotations (DRY principle)
def add_annotation(ax, value, pos_x, color='#114f7a'):
    """Add value annotation to chart"""
    if not np.isnan(value):
        ax.annotate(f'{value:.1f}', xy=(pos_x, value), xytext=(5, 5),
                   textcoords='offset points', fontsize=8, fontweight='bold',
                   bbox=dict(boxstyle='round,pad=0.2', facecolor=color, alpha=0.2))

# Create figure with subplots
fig = plt.figure(figsize=(14, 16))
gs = GridSpec(6, 3, height_ratios=[4, 1, 1.5, 1.5, 1.5, 2], hspace=0.4, wspace=0.3)

# Reusable index and last position
idx, last_idx = data.index, data.index[-1]

# Main price chart with moving averages and Bollinger Bands
ax_main = fig.add_subplot(gs[0, :])
ax_main.plot(idx, close_price, color="#114f7a", label=f'{selected_stock} Close', linewidth=2.5)
ax_main.plot(idx, data['MA_20'], color="#114e7ae0", alpha=0.8, linestyle='--', label='20-day MA', linewidth=1.5)
ax_main.plot(idx, data['MA_50'], color="#114e7ac8", alpha=0.8, linestyle=':', label='50-day MA', linewidth=1.5)
ax_main.fill_between(idx, lower_band, upper_band, color='gray', alpha=0.2, label='Bollinger Bands')
ax_main.set_title(f'{selected_stock} Stock Price with Moving Averages & Bollinger Bands ({period_display})', 
                  fontsize=16, fontweight='bold', pad=20)
ax_main.set_ylabel('Price ($)', fontsize=12)
ax_main.legend(fontsize=10, loc='upper left')
ax_main.grid(True, alpha=0.3)
ax_main.tick_params(labelbottom=False)
ax_main.annotate(f'${close_price.iloc[-1]:.2f}', xy=(last_idx, close_price.iloc[-1]), xytext=(10, 10),
                textcoords='offset points', bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.7),
                fontsize=10, fontweight='bold')

# Volume chart (optimized with numpy operations)
ax_volume = fig.add_subplot(gs[1, :], sharex=ax_main)
volume_data = data['Volume'] / 1e6
colors = np.where(close_price.diff() > 0, '#00ff00', '#ff0000')
colors[0] = '#d62728'  # First bar gets default color
ax_volume.bar(idx, volume_data, color=colors, alpha=0.6, width=1)
ax_volume.set_ylabel('Volume\n(Millions)', fontsize=10)
ax_volume.grid(True, alpha=0.3)
ax_volume.tick_params(labelsize=9)

# RSI chart
ax_rsi = fig.add_subplot(gs[2, 0])
ax_rsi.plot(idx, data['RSI'], color='#114f7a', linewidth=2, alpha=0.8)
ax_rsi.axhline(y=70, color='red', linestyle='--', alpha=0.7, label='Overbought (70)')
ax_rsi.axhline(y=30, color='green', linestyle='--', alpha=0.7, label='Oversold (30)')
ax_rsi.set_ylim(0, 100)
ax_rsi.set_title('RSI', fontweight='bold', fontsize=11)
ax_rsi.set_ylabel('RSI', fontsize=9)
ax_rsi.legend(fontsize=7)
ax_rsi.tick_params(axis='both', labelsize=8)
add_annotation(ax_rsi, data['RSI'].iloc[-1], last_idx, '#9467bd')

# Volatility chart
ax_vol = fig.add_subplot(gs[2, 1])
volatility_pct = data['Volatility'] * 100
ax_vol.plot(idx, volatility_pct, color='#114f7a', linewidth=2, alpha=0.8)
ax_vol.set_title('Volatility', fontweight='bold', fontsize=11)
ax_vol.set_ylabel('Volatility (%)', fontsize=9)
ax_vol.tick_params(axis='both', labelsize=8)
add_annotation(ax_vol, volatility_pct.iloc[-1], last_idx)

# Cumulative Returns chart
ax_cum = fig.add_subplot(gs[2, 2])
cumulative_returns_pct = data['Cumulative_Return'] * 100
ax_cum.plot(idx, cumulative_returns_pct, color='#114f7a', linewidth=2, alpha=0.8)
ax_cum.set_title('Cumulative Returns', fontweight='bold', fontsize=11)
ax_cum.set_ylabel('Cumulative Returns (%)', fontsize=9)
ax_cum.tick_params(axis='both', labelsize=8)
add_annotation(ax_cum, cumulative_returns_pct.iloc[-1], last_idx)

# MACD chart
ax_macd = fig.add_subplot(gs[3, :2])
ax_macd.plot(idx, macd, label='MACD', color='blue', linewidth=1.5)
ax_macd.plot(idx, signal, label='Signal', color='red', linewidth=1)
ax_macd.bar(idx, histogram, color='gray', alpha=0.5, width=1)
ax_macd.axhline(y=0, color='black', linestyle='--', alpha=0.5)
ax_macd.set_title('MACD', fontweight='bold', fontsize=11)
ax_macd.legend(fontsize=8)
ax_macd.tick_params(axis='both', labelsize=8)

# Maximum Drawdown chart
ax_dd = fig.add_subplot(gs[3, 2])
ax_dd.fill_between(idx, drawdown, 0, color='red', alpha=0.3)
ax_dd.set_title('Max Drawdown (%)', fontweight='bold', fontsize=11)
ax_dd.set_ylabel('Drawdown (%)', fontsize=9)
ax_dd.tick_params(axis='both', labelsize=8)
add_annotation(ax_dd, drawdown.iloc[-1], last_idx, 'red')

# Stochastic Oscillator
ax_stoch = fig.add_subplot(gs[4, 0])
ax_stoch.plot(idx, k_percent, label='%K', color='blue')
ax_stoch.plot(idx, d_percent, label='%D', color='red')
ax_stoch.axhline(y=80, color='red', linestyle='--', alpha=0.7, label='Overbought')
ax_stoch.axhline(y=20, color='green', linestyle='--', alpha=0.7, label='Oversold')
ax_stoch.set_title('Stochastic Oscillator', fontweight='bold', fontsize=11)
ax_stoch.legend(fontsize=7)
ax_stoch.tick_params(axis='both', labelsize=8)

# Returns Distribution Histogram
ax_hist = fig.add_subplot(gs[4, 1])
ax_hist.hist(daily_returns.dropna() * 100, bins=50, alpha=0.7, color='purple', edgecolor='black')
ax_hist.set_title('Daily Returns Distribution', fontweight='bold', fontsize=11)
ax_hist.set_xlabel('Return (%)')
ax_hist.axvline(x=0, color='red', linestyle='--')
ax_hist.tick_params(axis='both', labelsize=8)

# On-Balance Volume
ax_obv = fig.add_subplot(gs[4, 2])
ax_obv.plot(idx, obv / 1e6, color='orange', linewidth=2)
ax_obv.set_title('On-Balance Volume (Millions)', fontweight='bold', fontsize=11)
ax_obv.grid(True, alpha=0.3)
ax_obv.tick_params(axis='both', labelsize=8)

# Rolling Sharpe Ratio
ax_sharpe_roll = fig.add_subplot(gs[5, :])
ax_sharpe_roll.plot(idx, rolling_sharpe, color='green', linewidth=2)
ax_sharpe_roll.set_title('Rolling Sharpe Ratio (60-day)', fontweight='bold', fontsize=10)
ax_sharpe_roll.axhline(y=0, color='black', linestyle='--', alpha=0.5)
ax_sharpe_roll.grid(True, alpha=0.3)
ax_sharpe_roll.tick_params(axis='both', labelsize=8)

# Set up proper date formatting for x-axis based on period
setup_date_formatting(ax_sharpe_roll, period)

plt.tight_layout()
plt.subplots_adjust(bottom=0.15)
plt.show()

In [ ]:
# Display key metrics with visual cards and mini charts
from IPython.display import HTML, display
from io import BytesIO
import base64
import matplotlib.pyplot as plt
from functions import humanize_number, safe_float, get_current_price_and_date

# ============================================================================
# CHART CREATION
# ============================================================================

def create_mini_chart(values, current_value, color='#3b82f6'):
    """Create a mini column chart showing 4 bars."""
    if not values or all(v is None for v in values):
        return None
    
    # Get last 4 values and pad if needed
    values = values[-4:]
    values = [None] * (4 - len(values)) + values
    
    fig, ax = plt.subplots(figsize=(1.2, 0.7))
    fig.patch.set_alpha(0)
    ax.set_facecolor('none')
    
    # Create bars
    bar_values = [v if v else 0 for v in values]
    bars = ax.bar(range(4), bar_values, color=color, width=0.7)
    
    # Style bars
    for bar in bars:
        bar.set_capstyle('round')
        bar.set_joinstyle('round')
    
    # Remove all axes
    ax.axis('off')
    plt.tight_layout(pad=0)
    
    # Convert to base64
    buf = BytesIO()
    plt.savefig(buf, format='png', dpi=50, bbox_inches='tight', transparent=True)
    plt.close(fig)
    buf.seek(0)
    
    return f'data:image/png;base64,{base64.b64encode(buf.read()).decode()}'

# ============================================================================
# HTML CARD CREATION
# ============================================================================

def create_metric_card(name, value, change_pct=None, chart_img=None, subtitle=None):
    """Create HTML for a metric card."""
    # Build change indicator
    change_html = ''
    if change_pct is not None:
        color = '#10b981' if change_pct > 0 else '#ef4444' if change_pct < 0 else '#6b7280'
        symbol = '▲' if change_pct > 0 else '▼' if change_pct < 0 else '●'
        change_html = f'<div style="color: {color}; font-size: 12px; margin-top: 4px;">{symbol} {abs(change_pct):.1f}%</div>'
    
    subtitle_html = f'<div style="color: #9ca3af; font-size: 11px; margin-top: 2px;">{subtitle}</div>' if subtitle else ''
    
    # Base card style
    card_style = '''background: white; border: 1px solid #e5e7eb; border-radius: 8px; 
                    padding: 16px; min-width: 180px; box-shadow: 0 1px 3px rgba(0,0,0,0.1);'''
    
    content = f'''
        <div style="color: #6b7280; font-size: 13px; font-weight: 500; margin-bottom: 8px;">{name}</div>
        <div style="color: #111827; font-size: 24px; font-weight: 700;">{value}</div>
        {subtitle_html}{change_html}
    '''
    
    if chart_img:
        return f'''<div style="{card_style} display: flex; justify-content: space-between; align-items: center; gap: 12px;">
            <div style="flex: 1;">{content}</div>
            <div style="flex-shrink: 0;"><img src="{chart_img}" style="width: 70px; height: 40px;"></div>
        </div>'''
    
    return f'<div style="{card_style}">{content}</div>'

def create_performance_card(name, change_pct):
    """Create HTML for a performance card with colored value."""
    if change_pct is None:
        return create_metric_card(name, 'N/A')
    
    color = '#10b981' if change_pct > 0 else '#ef4444' if change_pct < 0 else '#6b7280'
    symbol = '▲' if change_pct > 0 else '▼' if change_pct < 0 else '●'
    
    # Determine format
    is_percentage = any(keyword in name for keyword in ['%', 'Return', 'Volatility', 'Drawdown', 'VaR'])
    formatted_value = f'{symbol} {abs(change_pct):.1f}%' if is_percentage else f'{symbol} {abs(change_pct):.2f}'
    
    return f'''<div style="background: white; border: 1px solid #e5e7eb; border-radius: 8px; 
                           padding: 16px; min-width: 180px; box-shadow: 0 1px 3px rgba(0,0,0,0.1);">
        <div style="color: #6b7280; font-size: 13px; font-weight: 500; margin-bottom: 8px;">{name}</div>
        <div style="color: {color}; font-size: 24px; font-weight: 700;">{formatted_value}</div>
    </div>'''

# ============================================================================
# DATA EXTRACTION & CALCULATIONS
# ============================================================================

def extract_quarterly_data(data):
    """Extract quarterly historical values efficiently."""
    if not isinstance(data, pd.DataFrame) or data.empty:
        return {}, None
    
    # Ensure index is datetime
    if not isinstance(data.index, pd.DatetimeIndex):
        return {}, None
    
    quarterly_data = data.resample('Q').last()
    if len(quarterly_data) >= 5:
        quarterly_data = quarterly_data.iloc[-5:-1]  # last 4 completed quarters
    
    result = {}
    for col in ['Close', 'RSI', 'Volume']:
        if col in quarterly_data.columns:
            result[col] = quarterly_data[col].tolist()
    
    return result, quarterly_data

def calculate_performance_metrics(data):
    """Calculate all performance metrics at once."""
    metrics = {}
    if not isinstance(data, pd.DataFrame) or data.empty or 'Close' not in data.columns:
        return metrics
    
    close_prices = data['Close']
    periods = {'30d': 30, '90d': 90, '6m': 126, '1y': 252}
    
    for name, days in periods.items():
        if len(data) >= days:
            metrics[name] = ((close_prices.iloc[-1] - close_prices.iloc[-days]) / close_prices.iloc[-days]) * 100
        elif name == '1y' and len(data) > 126:
            metrics[name] = ((close_prices.iloc[-1] - close_prices.iloc[0]) / close_prices.iloc[0]) * 100
    
    return metrics

def calculate_risk_metrics(data):
    """Calculate all risk metrics at once."""
    metrics = {k: None for k in ['sharpe_ratio', 'sortino_ratio', 'calmar_ratio', 
                                   'annualized_volatility', 'annualized_return', 'max_drawdown', 'var_95']}
    
    if not isinstance(data, pd.DataFrame) or data.empty or 'Close' not in data.columns or len(data) < 30:
        return metrics
    
    returns = data['Close'].pct_change().dropna()
    if returns.empty:
        return metrics
    
    # Annualized metrics
    total_return = (data['Close'].iloc[-1] / data['Close'].iloc[0]) - 1
    years = len(data) / 252
    if years > 0:
        metrics['annualized_return'] = ((1 + total_return) ** (1 / years) - 1) * 100
    
    metrics['annualized_volatility'] = returns.std() * np.sqrt(252) * 100
    
    # Sharpe Ratio (assuming risk-free rate of 0 for simplicity)
    if metrics['annualized_volatility'] and metrics['annualized_volatility'] > 0:
        metrics['sharpe_ratio'] = (metrics['annualized_return'] or 0) / metrics['annualized_volatility']
    
    # Max Drawdown
    cumulative = (1 + returns).cumprod()
    running_max = cumulative.expanding().max()
    drawdown = (cumulative - running_max) / running_max
    metrics['max_drawdown'] = drawdown.min() * 100
    
    # Sortino Ratio
    downside_returns = returns[returns < 0]
    if len(downside_returns) > 0:
        downside_std = downside_returns.std() * np.sqrt(252)
        if downside_std > 0:
            metrics['sortino_ratio'] = (metrics['annualized_return'] or 0) / (downside_std * 100)
    
    # Calmar Ratio
    if metrics['max_drawdown'] and metrics['max_drawdown'] < 0 and metrics['annualized_return']:
        metrics['calmar_ratio'] = metrics['annualized_return'] / abs(metrics['max_drawdown'])
    
    # VaR (95%)
    metrics['var_95'] = abs(returns.quantile(0.05)) * 100
    
    return metrics

# ============================================================================
# MAIN EXECUTION
# ============================================================================

if 'selected_stock' not in globals() or not selected_stock:
    display(HTML('<div style="color: #dc2626; font-weight: 600;">No stock selected. Set user_input and run the STOCK SELECTION cell first.</div>'))
else:
    # Initialize ticker
    try:
        ticker = yf.Ticker(selected_stock)
    except Exception as e:
        display(HTML(f'<div style="color: #dc2626;">Error creating ticker for {selected_stock}: {e}</div>'))
        raise
    
    # Fetch all data - use the 'data' variable that was already loaded
    recent_hist = ticker.history(period='5d') if ticker else None
    
    # Use existing 'data' variable from previous cells
    if 'data' not in globals() or data is None or (isinstance(data, pd.DataFrame) and data.empty):
        data = ticker.history(period='2y') if ticker else None
        if isinstance(data, pd.DataFrame) and not data.empty:
            from functions import calculate_technical_indicators
            data = calculate_technical_indicators(data)
    
    info = getattr(ticker, 'info', {}) or {}
    quarterly_earnings_dates = getattr(ticker, 'earnings_dates', None)
    quarterly_eps = (quarterly_earnings_dates['EPS'].dropna().tail(4).tolist() 
                     if quarterly_earnings_dates is not None and not quarterly_earnings_dates.empty 
                     and 'EPS' in quarterly_earnings_dates.columns else [])
    
    # Extract quarterly data
    quarterly_values, quarterly_data = extract_quarterly_data(data)
    hist_values_price = quarterly_values.get('Close', [])
    hist_values_rsi = quarterly_values.get('RSI', [])
    hist_values_volume = quarterly_values.get('Volume', [])
    
    # Get current price
    current_price, last_date = get_current_price_and_date(recent_hist, data, info)
    
    # Calculate 1-day price change
    price_change_1d = None
    if isinstance(recent_hist, pd.DataFrame) and len(recent_hist) >= 2:
        p0, p1 = safe_float(recent_hist['Close'].iloc[-2]), safe_float(recent_hist['Close'].iloc[-1])
        if p0:
            price_change_1d = (p1 - p0) / p0 * 100.0
    
    # Extract info metrics with proper dividend yield handling
    dividend_yield_raw = info.get('dividendYield')
    # yfinance returns dividend yield as a decimal (0.025 = 2.5%)
    # Only multiply by 100 if the value is less than 1 (to convert decimal to percentage)
    if dividend_yield_raw is not None:
        # If value is already > 1, assume it's already in percentage
        if dividend_yield_raw < 1:
            dividend_yield_pct = dividend_yield_raw * 100
        else:
            dividend_yield_pct = dividend_yield_raw
    else:
        dividend_yield_pct = None
    
    metrics_from_info = {
        'market_cap': info.get('marketCap'),
        'pe_ratio': info.get('trailingPE'),
        'eps': info.get('trailingEps'),
        'pb_ratio': info.get('priceToBook'),
        'dividend_yield': dividend_yield_pct,
        'week_52_high': info.get('fiftyTwoWeekHigh'),
        'week_52_low': info.get('fiftyTwoWeekLow'),
        'volume': info.get('volume'),
        'beta': info.get('beta'),
        'ebitda': info.get('ebitda'),
        'book_value': info.get('bookValue'),
        'company_name': info.get('longName') or info.get('shortName') or selected_stock,
        'sector': info.get('sector', 'N/A'),
        'industry': info.get('industry', 'N/A')
    }
    
    # Calculate technical indicators - use correct column names
    tech_indicators = {}
    if isinstance(data, pd.DataFrame):
        # Map technical indicators to their actual column names in the data
        indicator_mapping = [
            ('rsi', 'RSI'),
            ('macd', 'MACD'),
            ('stochastic_k', '%K'),  # This is already 0-100 range from calculate_technical_indicators
            ('volume', 'Volume')
        ]
        
        for indicator, column in indicator_mapping:
            if column in data.columns:
                series = data[column].dropna()
                if not series.empty:
                    tech_indicators[indicator] = safe_float(series.iloc[-1])
        
        # Bollinger %B - use correct column names
        if all(col in data.columns for col in ['Close', 'Upper_Band', 'Lower_Band']):
            close_val = safe_float(data['Close'].iloc[-1])
            upper = safe_float(data['Upper_Band'].iloc[-1])
            lower = safe_float(data['Lower_Band'].iloc[-1])
            if close_val and upper and lower and upper != lower:
                tech_indicators['bollinger_b'] = (close_val - lower) / (upper - lower)
    
    # Calculate performance and risk metrics
    performance = calculate_performance_metrics(data)
    risk = calculate_risk_metrics(data)
    
    # ============================================================================
    # CALCULATE CHANGES & CHARTS
    # ============================================================================
    
    # Calculate 1-year changes
    changes = {}
    if metrics_from_info['market_cap'] and hist_values_price and current_price and current_price != 0:
        if isinstance(data, pd.DataFrame) and not data.empty and len(data) >= 250:
            price_1y = data['Close'].iloc[-250]
            market_cap_1y = price_1y * (metrics_from_info['market_cap'] / current_price)
        elif len(hist_values_price) > 0:
            market_cap_1y = hist_values_price[0] * (metrics_from_info['market_cap'] / current_price)
        else:
            market_cap_1y = None
        
        if market_cap_1y and market_cap_1y != 0:
            changes['market_cap'] = ((metrics_from_info['market_cap'] - market_cap_1y) / market_cap_1y) * 100
    
    if metrics_from_info['eps'] and quarterly_eps and len(quarterly_eps) > 0 and quarterly_eps[0] != 0:
        changes['eps'] = ((metrics_from_info['eps'] - quarterly_eps[0]) / quarterly_eps[0]) * 100
    
    # Create charts
    charts = {}
    if hist_values_price and current_price:
        charts['price'] = create_mini_chart(hist_values_price, current_price)
    
    if metrics_from_info['market_cap'] and hist_values_price and current_price and current_price != 0:
        quarterly_market_cap = [p * (metrics_from_info['market_cap'] / current_price) for p in hist_values_price]
        charts['market_cap'] = create_mini_chart(quarterly_market_cap, metrics_from_info['market_cap'])
    
    if metrics_from_info['pe_ratio'] and quarterly_eps:
        quarterly_pe = [current_price / eps for eps in quarterly_eps if eps and eps != 0]
        if quarterly_pe:
            charts['pe'] = create_mini_chart(quarterly_pe, metrics_from_info['pe_ratio'])
    
    if metrics_from_info['eps'] and quarterly_eps:
        charts['eps'] = create_mini_chart(quarterly_eps, metrics_from_info['eps'])
    
    if metrics_from_info['pb_ratio'] and hist_values_price and metrics_from_info['book_value'] and metrics_from_info['book_value'] != 0:
        quarterly_pb = [p / metrics_from_info['book_value'] for p in hist_values_price]
        charts['pb'] = create_mini_chart(quarterly_pb, metrics_from_info['pb_ratio'])
    
    if tech_indicators.get('rsi') and hist_values_rsi:
        charts['rsi'] = create_mini_chart(hist_values_rsi, tech_indicators['rsi'])
    
    if tech_indicators.get('volume') and hist_values_volume:
        charts['volume'] = create_mini_chart(hist_values_volume, tech_indicators['volume'])
    
    # ============================================================================
    # BUILD HTML DISPLAY
    # ============================================================================
    
    html_parts = [f'''
    <div style="font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, sans-serif; padding: 20px;">
        <h2 style="color: #111827; margin-bottom: 8px;">{metrics_from_info['company_name']} ({selected_stock})</h2>
        <div style="color: #6b7280; font-size: 14px; margin-bottom: 24px;">
            {metrics_from_info['sector']} • {metrics_from_info['industry']}
        </div>
    ''']
    
    # Main metrics grid
    html_parts.append('<div style="display: grid; grid-template-columns: repeat(auto-fill, minmax(180px, 1fr)); gap: 16px; margin-bottom: 24px;">')
    
    # Define cards with configuration
    main_cards = [
        ('Current Price', f'${current_price:.2f}' if current_price else 'N/A', price_change_1d, charts.get('price')),
        ('Market Cap', f'${humanize_number(metrics_from_info["market_cap"])}' if metrics_from_info['market_cap'] else 'N/A', 
         changes.get('market_cap'), charts.get('market_cap')),
        ('P/E Ratio', f'{metrics_from_info["pe_ratio"]:.2f}' if metrics_from_info['pe_ratio'] else 'N/A', None, charts.get('pe')),
        ('EPS', f'${metrics_from_info["eps"]:.2f}' if metrics_from_info['eps'] else 'N/A', changes.get('eps'), charts.get('eps')),
        ('EBITDA', f'${humanize_number(metrics_from_info["ebitda"])}' if metrics_from_info['ebitda'] else 'N/A', None, None),
        ('Dividend Yield', f'{metrics_from_info["dividend_yield"]:.2f}%' if metrics_from_info['dividend_yield'] else 'N/A', None, None),
        ('P/B Ratio', f'{metrics_from_info["pb_ratio"]:.2f}' if metrics_from_info['pb_ratio'] else 'N/A', None, charts.get('pb')),
        ('52W High', f'${metrics_from_info["week_52_high"]:.2f}' if metrics_from_info['week_52_high'] else 'N/A', None, None),
        ('52W Low', f'${metrics_from_info["week_52_low"]:.2f}' if metrics_from_info['week_52_low'] else 'N/A', None, None),
        ('Volume', humanize_number(metrics_from_info['volume'] or tech_indicators.get('volume')) 
         if (metrics_from_info['volume'] or tech_indicators.get('volume')) else 'N/A', None, charts.get('volume')),
        ('RSI', f'{tech_indicators["rsi"]:.1f}' if tech_indicators.get('rsi') else 'N/A', None, charts.get('rsi')),
        ('MACD', f'{tech_indicators["macd"]:.3f}' if tech_indicators.get('macd') is not None else 'N/A', None, None),
        ('Bollinger %B', f'{tech_indicators["bollinger_b"]:.2f}' if tech_indicators.get('bollinger_b') is not None else 'N/A', None, None),
        ('Stochastic %K', f'{tech_indicators["stochastic_k"]:.1f}' if tech_indicators.get('stochastic_k') is not None else 'N/A', None, None),
        ('Beta', f'{metrics_from_info["beta"]:.2f}' if metrics_from_info['beta'] else 'N/A', None, None, '(vs S&P 500)'),
    ]
    
    for card_data in main_cards:
        subtitle = card_data[4] if len(card_data) > 4 else None
        html_parts.append(create_metric_card(card_data[0], card_data[1], card_data[2], card_data[3], subtitle))
    
    html_parts.append('</div>')  # Close main grid
    
    # Risk Metrics section
    html_parts.append('<h3 style="color: #111827; margin-top: 24px; margin-bottom: 16px;">Risk Metrics</h3>')
    html_parts.append('<div style="display: grid; grid-template-columns: repeat(auto-fill, minmax(180px, 1fr)); gap: 16px;">')
    
    risk_cards = [
        ('Sharpe Ratio', risk.get('sharpe_ratio')),
        ('Sortino Ratio', risk.get('sortino_ratio')),
        ('Calmar Ratio', risk.get('calmar_ratio')),
        ('Annualized Volatility', risk.get('annualized_volatility')),
        ('Annualized Return', risk.get('annualized_return')),
        ('Max Drawdown', risk.get('max_drawdown')),
        ('VaR (95%)', risk.get('var_95')),
    ]
    
    for name, value in risk_cards:
        html_parts.append(create_performance_card(name, value))
    
    html_parts.append('</div>')  # Close risk metrics grid
    
    # Performance section
    html_parts.append('<h3 style="color: #111827; margin-top: 24px; margin-bottom: 16px;">Performance</h3>')
    html_parts.append('<div style="display: grid; grid-template-columns: repeat(auto-fill, minmax(180px, 1fr)); gap: 16px;">')
    
    perf_cards = [
        ('30-Day Return', performance.get('30d')),
        ('90-Day Return', performance.get('90d')),
        ('6-Month Return', performance.get('6m')),
        ('1-Year Return', performance.get('1y')),
    ]
    
    for name, value in perf_cards:
        html_parts.append(create_performance_card(name, value))
    
    html_parts.append('</div></div>')  # Close performance grid and main container
    
    # Display
    display(HTML(''.join(html_parts)))


[Back to Top](#table-of-contents)

# Under construction - Future Enhancement

This section is planned for additional analysis features. Currently contains basic yfinance info display.

In [ ]:
# Display company officers
try:
    ticker = globals().get('ticker') or yf.Ticker(selected_stock)
    officers = ticker.info.get('companyOfficers')
    display_structured_data_as_markdown(officers, "Company Officers", selected_stock)
except Exception as e:
    display(Markdown(f"**Error displaying company officers:** {e}"))

In [ ]:
# Render Institutional Holdings as Markdown

try:
    if 'ticker' not in globals():
        ticker = yf.Ticker(selected_stock)
    info = ticker.info or {}
    shares_outstanding = info.get('sharesOutstanding')

    institutional_holders = ticker.institutional_holders
    if institutional_holders is not None and not institutional_holders.empty:
        md_lines = []
        md_lines.append(f"### Top Institutional Holdings for {selected_stock}")
        md_lines.append('')

        held_pct = info.get('heldPercentInstitutions', None)
        if held_pct is not None:
            md_lines.append(f"**Percent held by Institutions:** {held_pct*100:.2f}%")
        else:
            md_lines.append("**Percent held by Institutions:** N/A")

        md_lines.append('')
        # Build numbered list of top 10 holders
        for i, row in enumerate(institutional_holders.head(10).itertuples(index=False), 1):
            # row typically has attributes 'Holder' and 'Shares' but structure may vary
            try:
                holder = getattr(row, 'Holder', None) or getattr(row, 'Holder', None) or list(row)[0]
            except Exception:
                holder = 'Unknown'
            try:
                shares = getattr(row, 'Shares', None) or getattr(row, 'Shares', None) or (list(row)[1] if len(list(row)) > 1 else 0)
            except Exception:
                shares = 0

            # Format shares
            shares_str = 'N/A'
            if shares and shares != 0:
                try:
                    shares_val = float(shares)
                    shares_str = f"{humanize_number(shares_val)} shares"

                    if shares_outstanding and shares_outstanding > 0:
                        pct = (shares_val / shares_outstanding) * 100
                        shares_str += f" ({pct:.2f}%)"
                except Exception:
                    shares_str = str(shares)

            md_lines.append(f"{i}. **{holder}** — {shares_str}")

        display(Markdown('\n\n'.join(md_lines)))
    else:
        display(Markdown(f"**Institutional Holdings:** Not available for {selected_stock}"))
except Exception as e:
    display(Markdown(f"**Error fetching holdings data:** {e}"))

In [ ]:
# Render Mutual Fund Holdings as Markdown
from IPython.display import Markdown, display

try:
    info = ticker.info or {}
    shares_outstanding = info.get('sharesOutstanding')

    mutualfund_holders = ticker.mutualfund_holders
    if mutualfund_holders is not None and not mutualfund_holders.empty:
        md_lines = []
        md_lines.append(f"### Top Mutual Fund Holdings for {selected_stock}")
        md_lines.append('')

        md_lines.append('')
        # Build numbered list of top 10 mutual fund holders
        for i, row in enumerate(mutualfund_holders.head(10).itertuples(index=False), 1):
            try:
                holder = getattr(row, 'Holder', None) or list(row)[0]
            except Exception:
                holder = 'Unknown'
            try:
                shares = getattr(row, 'Shares', None) or (list(row)[1] if len(list(row)) > 1 else 0)
            except Exception:
                shares = 0

            shares_str = 'N/A'
            if shares and shares != 0:
                try:
                    shares_val = float(shares)
                    shares_str = f"{humanize_number(shares_val)} shares"

                    if shares_outstanding and shares_outstanding > 0:
                        pct = (shares_val / shares_outstanding) * 100
                        shares_str += f" ({pct:.2f}%)"
                except Exception:
                    shares_str = str(shares)

            md_lines.append(f"{i}. **{holder}** — {shares_str}")

        display(Markdown('\n\n'.join(md_lines)))
    else:
        display(Markdown(f"**Mutual Fund Holdings:** Not available for {selected_stock}"))
except Exception as e:
    display(Markdown(f"**Error fetching mutual fund holdings data:** {e}"))

In [ ]:
# Print shares outstanding for the selected stoc
info = ticker.info or {}
shares_out = info.get('sharesOutstanding')

print(f"Shares outstanding: {humanize_number(shares_out)}")
# pprint(info)  # Uncomment if you need the full info dict


## Company Information

In [ ]:
# Fetch and display competitors from CNN Business API
from functions import make_cnn_api_request

# requires `selected_stock` to be set (e.g. 'AAPL', 'GOOGL', etc)
api_url = f"https://production.dataviz.cnn.io/quote/competitors/{selected_stock}/10"  # Can change the number at the end to get more competitors if available
desired_count = 10

competitors_list = []

# Make API request using utility function
resp = make_cnn_api_request(api_url)

md_lines = []
try:
    if resp is None:
        md_lines.append("**No response received (network error) — cannot fetch competitors.**")
    elif resp.status_code != 200:
        md_lines.append(f"**Final status:** {resp.status_code}. Response (truncated):")
        md_lines.append('')
        md_lines.append(resp.text[:1000])
    else:
        try:
            data = resp.json()
        except ValueError as e:
            md_lines.append("**Failed to decode JSON from response**")
            md_lines.append('')
            md_lines.append(resp.text[:2000])
            data = None

        if data is not None:
            # Normalize list/dict shapes
            if isinstance(data, dict) and "competitors" in data:
                items = data["competitors"]
            elif isinstance(data, list):
                items = data
            else:
                items = []

            def normalize_symbol(s):
                if not s:
                    return ""
                s = str(s).upper().strip()
                return s.split("-")[0].split(".")[0]

            target = normalize_symbol(selected_stock)

            if items:
                for comp in items:
                    sym = comp.get("symbol") or comp.get("ticker") or comp.get("ticker_symbol") or ""
                    sym_norm = normalize_symbol(sym)
                    if sym_norm == target:
                        continue
                    name = comp.get("name") or comp.get("companyName") or "Unknown"
                    competitors_list.append({"symbol": sym_norm or sym or "N/A", "name": name, "raw": comp})
                    if len(competitors_list) >= desired_count:
                        break

                if competitors_list:
                    md_lines.append(f"### Top {len(competitors_list)} competitors for {selected_stock}")
                    md_lines.append('')
                    for i, c in enumerate(competitors_list, 1):
                        md_lines.append(f"{i}. **{c['symbol']}** — {c['name']}")
                else:
                    md_lines.append("**No competitors found after excluding the stock itself.**")
                    md_lines.append('')
                    md_lines.append(json.dumps(items, indent=2)[:4000])
            else:
                md_lines.append("**No items in API response — dumping JSON (truncated):**")
                md_lines.append('')
                md_lines.append(json.dumps(data, indent=2)[:4000])

    display(Markdown('\n\n'.join(md_lines)))
except Exception as e:
    display(Markdown(f"**Error fetching competitors:** {e}"))

[Back to Top](#table-of-contents)

In [ ]:
# Quarterly and Yearly Income Statement and Balance Sheet with values in Billions

fig, axes = plt.subplots(2, 2, figsize=(18, 10))

# Quarterly data
revenue_q = ticker.quarterly_income_stmt.loc["Total Revenue"]
income_q = ticker.quarterly_income_stmt.loc["Net Income"]
assets_q = ticker.quarterly_balance_sheet.loc["Total Assets"]
debt_q = ticker.quarterly_balance_sheet.loc["Total Debt"]

# First subplot: Quarterly Total Revenue and Net Income with Profit Margin
revenue_b = revenue_q / 1e9
income_b = income_q / 1e9
df_plot1 = pd.DataFrame({'Revenue': revenue_b, 'Net Income': income_b})
df_plot1 = df_plot1.sort_index()  # Sort by date ascending
df_plot1 = df_plot1.tail(4)  # Show only the 4 most recent quarters
df_plot1.plot(kind='bar', color=["#000000", "#2a5fff"], width=0.8, ax=axes[0,0])
axes[0,0].set_title('Quarterly Total Revenue and Net Income (in Billions) with Profit Margin')
axes[0,0].set_ylabel('Amount (Billions USD)')
axes[0,0].set_xticks(range(len(df_plot1)))
axes[0,0].set_xticklabels([f"Q{((d.month-1)//3)+1} {d.year}" for d in df_plot1.index], rotation=0)
axes[0,0].grid(axis='y', linestyle='--', alpha=0.7)

# Calculate and plot Profit Margin on secondary axis for first subplot
profit_margin_q = (df_plot1['Net Income'] / df_plot1['Revenue']) * 100
ax1_twin = axes[0,0].twinx()
profit_margin_line_color = '#fb950f'  # Hex color for profit margin line
profit_margin_text_color = "#000000"  # Hex color for profit margin text/labels
ax1_twin.plot(range(len(df_plot1)), profit_margin_q.values, color=profit_margin_line_color, marker='o', linewidth=2, label='Profit Margin (%)')
ax1_twin.set_ylabel('Profit Margin (%)', color=profit_margin_text_color)
ax1_twin.tick_params(axis='y', labelcolor=profit_margin_text_color)
ax1_twin.set_ylim(bottom=0)  # Start profit margin axis at 0

# Combine legends for first subplot
lines1, labels1 = axes[0,0].get_legend_handles_labels()
lines1_twin, labels1_twin = ax1_twin.get_legend_handles_labels()
axes[0,0].legend(lines1 + lines1_twin, labels1 + labels1_twin, title='Metrics', bbox_to_anchor=(1.05, 1), loc='upper left')

# Second subplot: Quarterly Total Assets and Total Debt with Debt-to-Asset Ratio
assets_b = assets_q / 1e9
debt_b = debt_q / 1e9
df_plot2 = pd.DataFrame({'Total Assets': assets_b, 'Total Debt': debt_b})
df_plot2 = df_plot2.sort_index()  # Sort by date ascending
df_plot2 = df_plot2.tail(4)  # Show only the 4 most recent quarters
df_plot2.plot(kind='bar', color=["#000000", "#2a5fff"], width=0.8, ax=axes[0,1])
axes[0,1].set_title('Quarterly Total Assets and Total Debt (in Billions) with Debt-to-Asset Ratio')
axes[0,1].set_ylabel('Amount (Billions USD)')
axes[0,1].set_xticks(range(len(df_plot2)))
axes[0,1].set_xticklabels([f"Q{((d.month-1)//3)+1} {d.year}" for d in df_plot2.index], rotation=0)
axes[0,1].grid(axis='y', linestyle='--', alpha=0.7)

# Calculate and plot Debt-to-Asset Ratio on secondary axis for second subplot (Debt / Assets)
debt_to_asset_ratio_q = df_plot2['Total Debt'] / df_plot2['Total Assets']
debt_to_asset_ratio_pct_q = debt_to_asset_ratio_q * 100  # Convert to percentage
ax2_twin = axes[0,1].twinx()
ratio_line_color = '#fb950f'  # Red color for debt ratio
ratio_text_color = "#000000"  # Red color for text
ax2_twin.plot(range(len(df_plot2)), debt_to_asset_ratio_pct_q.values, color=ratio_line_color, marker='o', linewidth=3, linestyle='-', label='Debt-to-Asset Ratio (%)')
ax2_twin.set_ylabel('Debt-to-Asset Ratio (%)', color=ratio_text_color)
ax2_twin.tick_params(axis='y', labelcolor=ratio_text_color)
ax2_twin.set_ylim(bottom=0, top=100)  # Set to 0-100% range

# Combine legends for second subplot
lines2, labels2 = axes[0,1].get_legend_handles_labels()
lines2_twin, labels2_twin = ax2_twin.get_legend_handles_labels()
axes[0,1].legend(lines2 + lines2_twin, labels2 + labels2_twin, title='Metrics', bbox_to_anchor=(1.05, 1), loc='upper left')

# Yearly data
revenue_y = ticker.income_stmt.loc["Total Revenue"]
income_y = ticker.income_stmt.loc["Net Income"]
assets_y = ticker.balance_sheet.loc["Total Assets"]
debt_y = ticker.balance_sheet.loc["Total Debt"]

# Third subplot: Annual Total Revenue and Net Income with Profit Margin
revenue_b = revenue_y / 1e9
income_b = income_y / 1e9
df_plot3 = pd.DataFrame({'Revenue': revenue_b, 'Net Income': income_b})
df_plot3 = df_plot3.sort_index()  # Sort by date ascending
df_plot3 = df_plot3.tail(4)  # Show only the 4 most recent years
df_plot3.plot(kind='bar', color=["#000000", "#2a5fff"], width=0.8, ax=axes[1,0])
axes[1,0].set_title('Annual Total Revenue and Net Income (in Billions) with Profit Margin')
axes[1,0].set_ylabel('Amount (Billions USD)')
axes[1,0].set_xticks(range(len(df_plot3)))
axes[1,0].set_xticklabels([d.year for d in df_plot3.index], rotation=0)
axes[1,0].grid(axis='y', linestyle='--', alpha=0.7)

# Calculate and plot Profit Margin on secondary axis for third subplot
profit_margin_y = (df_plot3['Net Income'] / df_plot3['Revenue']) * 100
ax3_twin = axes[1,0].twinx()
ax3_twin.plot(range(len(df_plot3)), profit_margin_y.values, color=profit_margin_line_color, marker='o', linewidth=2, label='Profit Margin (%)')
ax3_twin.set_ylabel('Profit Margin (%)', color=profit_margin_text_color)
ax3_twin.tick_params(axis='y', labelcolor=profit_margin_text_color)
ax3_twin.set_ylim(bottom=0)  # Start profit margin axis at 0

# Combine legends for third subplot
lines3, labels3 = axes[1,0].get_legend_handles_labels()
lines3_twin, labels3_twin = ax3_twin.get_legend_handles_labels()
axes[1,0].legend(lines3 + lines3_twin, labels3 + labels3_twin, title='Metrics', bbox_to_anchor=(1.05, 1), loc='upper left')

# Fourth subplot: Annual Total Assets and Total Debt with Debt-to-Asset Ratio
assets_b = assets_y / 1e9
debt_b = debt_y / 1e9
df_plot4 = pd.DataFrame({'Total Assets': assets_b, 'Total Debt': debt_b})
df_plot4 = df_plot4.sort_index()  # Sort by date ascending
df_plot4 = df_plot4.tail(4)  # Show only the 4 most recent years
df_plot4.plot(kind='bar', color=["#000000", "#2a5fff"], width=0.8, ax=axes[1,1])
axes[1,1].set_title('Annual Total Assets and Total Debt (in Billions) with Debt-to-Asset Ratio')
axes[1,1].set_ylabel('Amount (Billions USD)')
axes[1,1].set_xticks(range(len(df_plot4)))
axes[1,1].set_xticklabels([d.year for d in df_plot4.index], rotation=0)
axes[1,1].grid(axis='y', linestyle='--', alpha=0.7)

# Calculate and plot Debt-to-Asset Ratio on secondary axis for fourth subplot (Debt / Assets)
debt_to_asset_ratio_y = df_plot4['Total Debt'] / df_plot4['Total Assets']
debt_to_asset_ratio_pct_y = debt_to_asset_ratio_y * 100  # Convert to percentage
ax4_twin = axes[1,1].twinx()
ax4_twin.plot(range(len(df_plot4)), debt_to_asset_ratio_pct_y.values, color=ratio_line_color, marker='o', linewidth=3, linestyle='-', label='Debt-to-Asset Ratio (%)')
ax4_twin.set_ylabel('Debt-to-Asset Ratio (%)', color=ratio_text_color)
ax4_twin.tick_params(axis='y', labelcolor=ratio_text_color)
ax4_twin.set_ylim(bottom=0, top=100)  # Set to 0-100% range

# Combine legends for fourth subplot
lines4, labels4 = axes[1,1].get_legend_handles_labels()
lines4_twin, labels4_twin = ax4_twin.get_legend_handles_labels()
axes[1,1].legend(lines4 + lines4_twin, labels4 + labels4_twin, title='Metrics', bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()
plt.show()

In [ ]:
# Display annual balance sheet with all rows visible
bs = ticker.balance_sheet
bs = bs.applymap(humanize_number)
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', None, 'display.max_colwidth', None):
    display(bs)

In [ ]:
roster_holder = ticker.insider_roster_holders

# Select only numeric columns and apply humanize_number to them
numeric_cols = roster_holder.select_dtypes(include=[np.number]).columns
roster_holder[numeric_cols] = roster_holder[numeric_cols].applymap(humanize_number)

# Display the DataFrame
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', None, 'display.max_colwidth', None):
    display(roster_holder)

Change the time on the Position Direct Date to not display

In [ ]:
# Format insider_transactions with humanize_number
insider_trans = ticker.insider_transactions.head(10)
numeric_cols = insider_trans.select_dtypes(include=[np.number]).columns
insider_trans[numeric_cols] = insider_trans[numeric_cols].applymap(humanize_number)
insider_trans

In [ ]:
# Insider Transactions

ticker = yf.Ticker(selected_stock)
ins = None
try:
    ins = ticker.insider_transactions
except Exception as e:
    print('Could not fetch insider_transactions:', e)

if ins is None or ins.empty:
    print('No insider transactions available for', selected_stock)
else:
    df_ins = ins.copy()

    # Sort by date if available
    date_col = next((c for c in ['Start Date', 'FilingDate', 'Date', 'TransactionDate'] if c in df_ins.columns), None)
    if date_col:
        df_ins.sort_values(by=date_col, ascending=False, inplace=True)

    # Top 10 newest
    df_top = df_ins.head(10)

    markdown_lines = [f"### Insider Transactions for {selected_stock}\n"]

    for _, row in df_top.iterrows():
        insider = row.get('Insider', 'Unknown')
        title = row.get('Title', row.get('Position', '')).strip()
        text = row.get('Text', '')  # descriptive text
        shares = row.get('Shares', row.get('SharesTransacted', row.get('SharesChanged', None)))
        value = row.get('Value', row.get('ValueTransacted', None))

        # Format values
        shares_h = humanize_number(shares)
        value_h = humanize_number(value)
        date = row.get(date_col, '')
        date_str = f"{pd.to_datetime(date).date()}" if pd.notna(date) else 'Unknown date'

        # Combine title and insider name
        who = f"**{title}**, {insider}" if title else f"{insider}"

        # Construct Markdown line
        if text:
            action_desc = text.strip()
        else:
            action_desc = "Transaction"

        markdown_lines.append(
            f"- {date_str}: {who} **{action_desc}** ({shares_h} shares) for an overall value of **${value_h}**"
        )

    # Display as Markdown
    display(Markdown("\n".join(markdown_lines)))

In [ ]:
# Render latest news articles from yfinance as Markdown

# Render ticker.news as wrapped Markdown (one article per numbered item)
try:
    news = getattr(ticker, 'news', None)
    md_lines = []

    if not news:
        md_lines.append("**No news available for this stock.**")
    else:
        md_lines.append(f"### News for {selected_stock} — {len(news)} articles found")
        md_lines.append('')

        for i, article in enumerate(news, 1):
            try:
                # Extract content (news data is nested under 'content' key)
                content = article.get('content', {}) if isinstance(article, dict) else {}

                # Extract provider from nested structure
                provider_obj = content.get('provider', {}) if isinstance(content, dict) else {}
                provider = provider_obj.get('displayName', 'Unknown') if isinstance(provider_obj, dict) else 'Unknown'

                # Extract summary
                summary = content.get('summary') or content.get('description') or content.get('title') or 'No summary available'

                # Extract pubDate
                pub_date = content.get('pubDate') or content.get('displayTime') or article.get('pubDate') or article.get('date') or 'Unknown date'

                # Format date if it's in ISO format
                if pub_date and pub_date != 'Unknown date':
                    try:
                        from datetime import datetime
                        dt = datetime.fromisoformat(str(pub_date).replace('Z', '+00:00'))
                        pub_date = dt.strftime('%Y-%m-%d %H:%M:%S')
                    except Exception:
                        pub_date = str(pub_date)

                # Truncate summary to 150 characters for one-line display
                summary_short = summary[:150] + '...' if isinstance(summary, str) and len(summary) > 150 else summary

                # Build Markdown line: numbered item with provider, bolded, summary, and date in parentheses
                md_lines.append(f"{i}. **{provider}** — {summary_short} ({pub_date})")
            except Exception as e:
                md_lines.append(f"{i}. Error parsing article: {e}")

    display(Markdown('\n\n'.join(md_lines)))
except Exception as e:
    display(Markdown(f"**Error rendering news:** {e}"))

In [ ]:
# Take a closer look at this - Analyst upgrades and downgrades data
ticker.upgrades_downgrades.head(10)

## Analyst Price Targets Visualization

This chart combines the stock price history with analyst price target distributions from upgrades/downgrades data.

In [ ]:
# Plot Recommendations Summary as a column (bar) chart

# Ensure `ticker` and `selected_stock` exist (run stock selection cell first)
try:
    rec = ticker.recommendations_summary
except (NameError, AttributeError):
    display(Markdown('**`ticker` is not defined or has no recommendations_summary. Run the STOCK SELECTION cell first.**'))
    rec = None

if rec is None or (isinstance(rec, pd.DataFrame) and rec.empty):
    display(Markdown('**No recommendations_summary available for this ticker.**'))
else:
    try:
        # recommendations_summary is typically a DataFrame with columns like:
        # period, strongBuy, buy, hold, sell, strongSell
        # We need to use ONLY THE MOST RECENT row (latest period)
        
        if isinstance(rec, pd.DataFrame):
            # Get the most recent row (last row in the DataFrame)
            latest_row = rec.iloc[-1]
            
            recommendation_counts = {}
            
            # Check various column name formats
            col_mapping = {
                'strongBuy': 'Strong Buy',
                'strong_buy': 'Strong Buy',
                'Strong Buy': 'Strong Buy',
                'buy': 'Buy',
                'Buy': 'Buy',
                'hold': 'Hold',
                'Hold': 'Hold',
                'sell': 'Sell',
                'Sell': 'Sell',
                'strongSell': 'Strong Sell',
                'strong_sell': 'Strong Sell',
                'Strong Sell': 'Strong Sell'
            }
            
            for col in rec.columns:
                if col in col_mapping:
                    label = col_mapping[col]
                    # Get value from the latest row only
                    value = latest_row[col]
                    if pd.notna(value) and value > 0:
                        recommendation_counts[label] = value
            
            if not recommendation_counts:
                display(Markdown('**No valid recommendation data found in the most recent period.**'))
            else:
                # Create Series from the counts
                s = pd.Series(recommendation_counts)
                
                # Order labels in the preferred order
                preferred_order = ['Strong Buy', 'Buy', 'Hold', 'Sell', 'Strong Sell']
                present = [p for p in preferred_order if p in s.index]
                s = s.reindex(present)
                
                # Plot vertical column chart
                fig, ax = plt.subplots(figsize=(10, 6))
                
                # Define colors for each category
                colors = []
                for label in s.index:
                    if 'Strong Buy' in label:
                        colors.append('#00aa00')  # Dark green
                    elif 'Buy' in label:
                        colors.append('#66cc66')  # Light green
                    elif 'Hold' in label:
                        colors.append('#ffaa00')  # Orange
                    elif 'Sell' in label and 'Strong' not in label:
                        colors.append('#ff6666')  # Light red
                    elif 'Strong Sell' in label:
                        colors.append('#cc0000')  # Dark red
                    else:
                        colors.append('#7f7f7f')  # Gray
                
                bars = ax.bar(s.index, s.values, color=colors, edgecolor='black', alpha=0.85, width=0.6)
                
                # Get the period/date if available
                period_info = ""
                if 'period' in rec.columns:
                    period_info = f" (Period: {latest_row['period']})"
                
                ax.set_title(f'Current Analyst Recommendations — {selected_stock}{period_info}', fontsize=14, fontweight='bold')
                ax.set_ylabel('Number of Analysts', fontsize=12)
                ax.set_xlabel('Recommendation', fontsize=12)
                plt.xticks(rotation=45, ha='right')
                ax.grid(axis='y', alpha=0.3, linestyle='--')
                
                # Annotate values on top of bars
                for bar in bars:
                    h = bar.get_height()
                    if h > 0:
                        ax.annotate(f'{int(h)}', 
                                  xy=(bar.get_x() + bar.get_width() / 2, h), 
                                  xytext=(0, 3),
                                  textcoords='offset points', 
                                  ha='center', 
                                  va='bottom', 
                                  fontsize=11,
                                  fontweight='bold')
                
                plt.tight_layout()
                plt.show()
        else:
            display(Markdown(f'**Unexpected data type for recommendations_summary: {type(rec)}**'))
            
    except Exception as e:
        display(Markdown(f'**Error preparing recommendations plot:** {e}'))
        import traceback
        print(traceback.format_exc())

[Back to Top](#table-of-contents)

In [ ]:
# Fear and Greed Index Data
url = "https://production.dataviz.cnn.io/index/fearandgreed/graphdata"

headers_list = [
    {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
        "Accept": "application/json, text/javascript, */*; q=0.01",
        "Referer": "https://www.cnn.com/",
    },
    {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36",
        "Accept": "application/json, text/javascript, */*; q=0.01",
        "Referer": "https://www.cnn.com/",
    },
    {"User-Agent": "curl/7.88.1", "Accept": "*/*"},
]

session = requests.Session()
retries = Retry(total=2, backoff_factor=0.8, status_forcelist=[429, 500, 502, 503, 504])
session.mount("https://", HTTPAdapter(max_retries=retries))

resp = None
for h in headers_list:
    try:
        resp = session.get(url, headers=h, timeout=8)
        if resp.status_code == 200:
            break
    except requests.exceptions.RequestException:
        resp = None

if resp is None:
    print("No response received (network error)")
else:
    data = resp.json()

    metric_keys = [
        'fear_and_greed', 'fear_and_greed_historical', 'market_momentum_sp500',
        'market_momentum_sp125', 'stock_price_strength', 'stock_price_breadth',
        'put_call_options', 'market_volatility_vix', 'market_volatility_vix_50',
        'junk_bond_demand', 'safe_haven_demand'
    ]

    def parse_ts(ts):
        if ts is None:
            return pd.NaT
        try:
            if isinstance(ts, (int, float)):
                t = float(ts)
                if t > 1e12:
                    return pd.to_datetime(t, unit='ms', utc=True)
                elif t > 1e9:
                    return pd.to_datetime(t, unit='s', utc=True)
            if isinstance(ts, str):
                try:
                    return pd.to_datetime(ts)
                except Exception:
                    return pd.NaT
        except Exception:
            return pd.NaT
        return pd.NaT

    rows = []
    for mk in metric_keys:
        entry = data.get(mk) if isinstance(data, dict) else None
        score = None
        rating = None
        timestamp = None

        if isinstance(entry, dict):
            score = entry.get('score')
            rating = entry.get('rating')
            timestamp = entry.get('timestamp')
            if (score is None or rating is None or timestamp is None) and isinstance(entry.get('data'), list) and entry.get('data'):
                last = entry['data'][-1]
                if isinstance(last, dict):
                    score = score if score is not None else (last.get('y') or last.get('value'))
                    rating = rating if rating is not None else (last.get('rating') or last.get('label'))
                    timestamp = timestamp if timestamp is not None else last.get('x')
        rows.append({'metric': mk, 'score': score, 'rating': rating, 'timestamp': parse_ts(timestamp)})

    df = pd.DataFrame(rows).set_index('metric')
    # Try to convert score to numeric
    df['score'] = pd.to_numeric(df['score'], errors='coerce')

    display(df)

In [ ]:
# Fear and greed index - historical trend plot
url = "https://production.dataviz.cnn.io/index/fearandgreed/graphdata"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Accept": "application/json, text/javascript, */*; q=0.01",
    "Referer": "https://www.cnn.com/",
}

session = requests.Session()
retries = Retry(total=2, backoff_factor=0.8, status_forcelist=[429, 500, 502, 503, 504])
session.mount("https://", HTTPAdapter(max_retries=retries))

try:
    resp = session.get(url, headers=headers, timeout=8)
    resp.raise_for_status()
    data = resp.json()
except Exception as e:
    raise SystemExit(f"Error fetching data: {e}")

# --- Parse data ---
def parse_timestamp(ts):
    """Convert CNN timestamp to pandas datetime"""
    return pd.to_datetime(ts, unit='ms', utc=True, errors='coerce')

historical = data.get("fear_and_greed_historical", {}).get("data", [])
if not historical:
    raise SystemExit("No historical data available.")

df = pd.DataFrame([
    {"timestamp": parse_timestamp(item.get("x")), "score": item.get("y")}
    for item in historical if item.get("x") and item.get("y") is not None
])
df = df.dropna().sort_values("timestamp")
df = df[(df["score"] >= 0) & (df["score"] <= 100)]

# --- Plot ---
def get_color(value):
    if value <= 25: return '#DC2626'   # Extreme Fear
    if value <= 45: return '#F97316'   # Fear
    if value <= 55: return '#FCD34D'   # Neutral
    if value <= 75: return '#10B981'   # Greed
    return '#059669'                   # Extreme Greed

def get_rating(value):
    if value <= 25: return 'Extreme Fear'
    if value <= 45: return 'Fear'
    if value <= 55: return 'Neutral'
    if value <= 75: return 'Greed'
    return 'Extreme Greed'

fig, ax = plt.subplots(figsize=(18, 6))

# Main line
ax.plot(df['timestamp'], df['score'], color='#1e3a8a', linewidth=3, alpha=0.9, zorder=3)

# Color fill
for i in range(len(df) - 1):
    x = [df['timestamp'].iloc[i], df['timestamp'].iloc[i + 1]]
    y = [df['score'].iloc[i], df['score'].iloc[i + 1]]
    ax.fill_between(x, 0, y, color=get_color(np.mean(y)), alpha=0.3)

# Reference lines
for y, color in [(25, '#DC2626'), (45, '#F97316'), (55, '#FCD34D'), (75, '#10B981')]:
    ax.axhline(y=y, color=color, linestyle='--', alpha=0.5, linewidth=1.5)

# Latest value annotation
last = df.iloc[-1]
ax.scatter(last['timestamp'], last['score'], color=get_color(last['score']),
           s=200, edgecolors='white', linewidth=3, zorder=5)
ax.annotate(f"{last['score']:.1f}\n{get_rating(last['score'])}",
            xy=(last['timestamp'], last['score']), xytext=(20, 20),
            textcoords='offset points', fontsize=10, fontweight='bold',
            bbox=dict(boxstyle='round,pad=0.7', facecolor=get_color(last['score']),
                      edgecolor='white', alpha=0.9), color='white')

# Labels & style
ax.set_title("CNN Fear & Greed Index - Historical Trend", fontsize=16, fontweight='bold')
ax.set_xlabel("Date", fontsize=11)
ax.set_ylabel("Score (0–100)", fontsize=11)
ax.set_ylim(0, 100)
ax.grid(alpha=0.25, linestyle=':')
ax.tick_params(axis='x', rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Fetch share-price insights and display only list_summary.plain_text (if present)

# requires `selected_stock` to be set (e.g. 'AAPL', 'GOOGL', etc)
api_url = f"https://production.dataviz.cnn.io/insights/share_price/{selected_stock}"

# Make API request using utility function
resp = make_cnn_api_request(api_url)

try:
    if resp is None:
        display(Markdown("**No response received (network error) — cannot fetch insights.**"))
    elif resp.status_code != 200:
        # Show truncated response body for debugging
        display(Markdown(f"**Final status:** {resp.status_code}. Response (truncated):\n\n{resp.text[:1000]}"))
    else:
        try:
            data = resp.json()
        except ValueError:
            display(Markdown("**Failed to decode JSON from response**"))
            display(Markdown(resp.text[:2000]))
            data = None

        # Try to extract list_summary -> plain_text with a few common key-name fallbacks
        def extract_plain_text(obj):
            # Direct dict with expected key
            if not isinstance(obj, (dict, list)):
                return None
            # Common top-level keys
            candidates = ['list_summary', 'listSummary', 'listsummary']
            for key in candidates:
                if isinstance(obj, dict) and key in obj:
                    val = obj[key]
                    if isinstance(val, dict):
                        for pk in ('plain_text', 'plainText', 'plaintext'):
                            if pk in val and isinstance(val[pk], str):
                                return val[pk].strip()
                    if isinstance(val, str):
                        return val.strip()
            # Check common nested places (e.g., 'summary', 'data' lists, etc.)
            if isinstance(obj, dict):
                for v in obj.values():
                    res = extract_plain_text(v)
                    if res:
                        return res
            if isinstance(obj, list):
                for item in obj:
                    res = extract_plain_text(item)
                    if res:
                        return res
            return None

        if data is not None:
            plain = extract_plain_text(data)
            if plain:
                # Display only the extracted plaintext
                display(Markdown(plain))
            else:
                # Fallback: show small dump to help debugging
                display(Markdown("**No list_summary.plain_text found — dumping top-level JSON (truncated):**"))
                display(Markdown(json.dumps(data, indent=2)[:4000]))
except Exception as e:
    display(Markdown(f"**Error fetching insights:** {e}"))

In [ ]:
# Real-time stock price streaming via WebSocket (BTC-USD example)
#!pip install yfinance nest_asyncio

# Import modules
import nest_asyncio
import asyncio

# Allow nested async loops (needed for Jupyter)
nest_asyncio.apply()

# Define a message handler
def message_handler(message):
    # Each message is a dict with price updates
    if "id" in message and message["id"] == "BTC-USD":
        price = message.get("price")
        if price:
            print(f"BTC-USD: ${price:.2f}")

# Create async function for WebSocket 
async def stream_btc():
    async with yf.AsyncWebSocket(verbose=False) as ws:
        await ws.subscribe(["BTC-USD"])
        print("Connected to Yahoo Finance WebSocket. Streaming BTC-USD live...")
        await ws.listen(message_handler)

# Run it
#await stream_btc()         # Uncomment this line to start streaming (will run indefinitely)

In [ ]:
# Import required libraries for world map
import sys
import io
import folium
from concurrent.futures import ThreadPoolExecutor, as_completed

# Step 1: Define major stock indices with full country names (matching GeoJSON)
country_indices = {
    'United States of America': '^GSPC',      # S&P 500
    'United Kingdom': '^FTSE',                # FTSE 100
    'Germany': '^GDAXI',                      # DAX
    'France': '^FCHI',                        # CAC 40
    'Japan': '^N225',                         # Nikkei 225
    'China': '000001.SS',                     # Shanghai Composite
    'India': '^BSESN',                        # BSE Sensex
    'Brazil': '^BVSP',                        # Bovespa
    'Canada': '^GSPTSE',                      # S&P/TSX
    'Australia': '^AXJO',                     # ASX 200
    'South Korea': '^KS11',                   # KOSPI
    'Mexico': '^MXX',                         # IPC Mexico
    'South Africa': '^JN0U.JO',               # JSE All Share
    'Spain': '^IBEX',                         # IBEX 35
    'Italy': 'FTSEMIB.MI',                    # FTSE MIB
    'Netherlands': '^AEX',                    # AEX
    'Switzerland': '^SSMI',                   # SMI
    'Sweden': '^OMX',                         # OMX Stockholm
    'Hong Kong': '^HSI',                      # Hang Seng
    'Singapore': '^STI',                      # Straits Times
    'Taiwan': '^TWII',                        # Taiwan Weighted
    'Indonesia': '^JKSE',                     # Jakarta Composite
    'Malaysia': '^KLSE',                      # FTSE Bursa Malaysia KLCI
    'Thailand': '^SET.BK',                    # SET Index
    'Norway': '^OSEAX',                       # Oslo Børs All-Share
    'Denmark': '^OMXC25',                     # OMX Copenhagen 25
    'Finland': '^OMXH25',                     # OMX Helsinki 25
    'Belgium': '^BFX',                        # BEL 20
    'Austria': '^ATX',                        # ATX
    'Turkey': 'XU100.IS',                     # BIST 100
    'Poland': 'WIG20.WA',                     # WIG20
    'Israel': '^TA125.TA',                    # TA-125
    'Argentina': '^MERV',                     # MERVAL
    'Chile': '^IPSA',                         # S&P/CLX IPSA
    'New Zealand': '^NZ50',                   # S&P/NZX 50
}

# Suppress all warnings and yfinance output
warnings.filterwarnings('ignore')

# Fetch single stock data with suppressed output
def fetch_single_stock(country, ticker_symbol):
    try:
        # Redirect stderr to suppress yfinance error messages
        old_stderr = sys.stderr
        sys.stderr = io.StringIO()
        
        stock = yf.Ticker(ticker_symbol)
        hist = stock.history(period='5d')
        
        # Restore stderr
        sys.stderr = old_stderr
        
        if len(hist) >= 2:
            latest_close = hist['Close'].iloc[-1]
            previous_close = hist['Close'].iloc[-2]
            daily_change = ((latest_close - previous_close) / previous_close) * 100
            
            return {
                'country': country,
                'ticker': ticker_symbol,
                'daily_change': daily_change,
                'close': latest_close
            }
    except Exception as e:
        # Restore stderr in case of exception
        sys.stderr = old_stderr
    return None

# Fetch stock data in parallel
def get_daily_changes():
    data = []
    
    # Use ThreadPoolExecutor for parallel requests
    with ThreadPoolExecutor(max_workers=10) as executor:
        # Submit all tasks
        future_to_country = {
            executor.submit(fetch_single_stock, country, ticker_symbol): country 
            for country, ticker_symbol in country_indices.items()
        }
        
        # Collect results as they complete
        for future in as_completed(future_to_country):
            result = future.result()
            if result is not None:
                data.append(result)
    
    return pd.DataFrame(data)

# Get the data
print("Fetching stock market data...")
df = get_daily_changes()
print(f"Successfully fetched data for {len(df)} countries")

# Create the Folium map with choropleth
def create_world_map(df):
    # Create base map (no zoom controls)
    m = folium.Map(
        location=[20, 0],
        zoom_start=2,
        tiles='CartoDB positron',
        zoom_control=False,
        scrollWheelZoom=False,
        doubleClickZoom=False,
        boxZoom=False,
        keyboard=False
    )
    
    # Get world GeoJSON
    world_geo = requests.get(
        'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/world-countries.json'
    ).json()
    
    # Create choropleth layer with custom colors
    folium.Choropleth(
        geo_data=world_geo,
        name='Stock Index Daily Change',
        data=df,
        columns=['country', 'daily_change'],
        key_on='feature.properties.name',
        fill_color='RdYlGn',
        fill_opacity=0.8,
        line_opacity=0.3,
        legend_name='Daily Change (%)',
        nan_fill_color='lightgray',
        nan_fill_opacity=0.4,
    ).add_to(m)
    
    # Add individual GeoJson layers for each country with popups
    for idx, row in df.iterrows():
        # Find the country in GeoJSON
        for feature in world_geo['features']:
            if feature['properties']['name'] == row['country']:
                # Create popup text
                popup_text = f"""
                <div style='font-family: Arial; width: 200px;'>
                    <b style='font-size: 14px;'>{row['country']}</b><br>
                    <hr style='margin: 5px 0;'>
                    <b>Index:</b> {row['ticker']}<br>
                    <b>Daily Change:</b> <span style='color: {"green" if row["daily_change"] >= 0 else "red"}; font-size: 16px; font-weight: bold;'>{row['daily_change']:+.2f}%</span><br>
                    <b>Close:</b> {row['close']:.2f}
                </div>
                """

                # Get country color
                if row['daily_change'] <= -2:
                    color = '#8B0000'  # Dark red
                elif row['daily_change'] <= -1:
                    color = '#FF0000'  # Red
                elif row['daily_change'] < 0:
                    color = '#FF8C00'  # Orange
                elif row['daily_change'] <= 1:
                    color = '#FFFF00'  # Yellow
                elif row['daily_change'] <= 2:
                    color = '#90EE90'  # Light green
                else:
                    color = '#006400'  # Dark green

                # Create individual GeoJson for this country
                country_geo = {
                    'type': 'FeatureCollection',
                    'features': [feature]
                }

                folium.GeoJson(
                    country_geo,
                    style_function=lambda x, color=color: {
                        'fillColor': color,
                        'color': 'black',
                        'weight': 1,
                        'fillOpacity': 0.8
                    },
                    popup=folium.Popup(popup_text, max_width=250),
                    name=row['country']
                ).add_to(m)
                break
    
    folium.LayerControl().add_to(m)
    return m

# Create and save the map
print("Creating world map...")
world_map = create_world_map(df)
world_map.save('stock_index_world_map.html')
print("Map saved as 'stock_index_world_map.html'")

# Display in Jupyter
world_map

## Options

[Back to Top](#table-of-contents)

In [ ]:
expirations = ticker.options
print(expirations)

expiry = expirations[0]  # Take the nearest expiry
option_chain = ticker.option_chain(expiry)

calls = option_chain.calls
puts = option_chain.puts

print(calls.head())
print(puts.head())

In [ ]:
put_call_ratio = puts['openInterest'].sum() / calls['openInterest'].sum()
print(f"Put/Call Open Interest Ratio: {put_call_ratio:.2f}")

In [ ]:
max_call_strike = calls.loc[calls['openInterest'].idxmax(), 'strike']
max_put_strike = puts.loc[puts['openInterest'].idxmax(), 'strike']
print(f"Max Call OI Strike: {max_call_strike}")
print(f"Max Put OI Strike: {max_put_strike}")

In [ ]:
atm_strike = calls.iloc[(calls['strike'] - ticker.history(period="1d")['Close'].iloc[-1]).abs().argsort()[:1]]['strike'].values[0]
atm_call = calls[calls['strike'] == atm_strike]['lastPrice'].values[0]
atm_put = puts[puts['strike'] == atm_strike]['lastPrice'].values[0]
expected_move = atm_call + atm_put
print(f"Approx. Expected Move by {expiry}: ±${expected_move:.2f}")

In [ ]:
iv_data = []
for exp in expirations[:5]:  # Check first few expirations
    oc = ticker.option_chain(exp)
    iv = oc.calls['impliedVolatility'].mean()
    iv_data.append({'expiry': exp, 'avg_IV': iv})

iv_df = pd.DataFrame(iv_data)
print(iv_df)

In [ ]:
expiry = ticker.options[0]  # nearest expiry
chain = ticker.option_chain(expiry)
calls, puts = chain.calls, chain.puts
spot = ticker.history(period="1d")['Close'].iloc[-1]

# Create subplots
fig, axs = plt.subplots(3, 1, figsize=(12, 12))

# First subplot: Open Interest by Strike
axs[0].bar(calls['strike'], calls['openInterest'], width=1.0, label='Calls OI', color='green', alpha=0.6)
axs[0].bar(puts['strike'], puts['openInterest'], width=1.0, label='Puts OI', color='red', alpha=0.6)
axs[0].axvline(spot, color='blue', linestyle='--', label=f'Spot Price (${spot:.2f})')
axs[0].set_title(f'Open Interest by Strike — {expiry}')
axs[0].set_xlabel('Strike Price')
axs[0].set_ylabel('Open Interest')
axs[0].legend()

# Second subplot: Put/Call Open Interest Ratio by Expiry
ratios = []
for exp in ticker.options[:6]:
    oc = ticker.option_chain(exp)
    pcr = oc.puts['openInterest'].sum() / oc.calls['openInterest'].sum()
    ratios.append({'expiry': exp, 'put_call_ratio': pcr})

df_ratios = pd.DataFrame(ratios)
axs[1].plot(df_ratios['expiry'], df_ratios['put_call_ratio'], marker='o')
axs[1].set_title('Put/Call Open Interest Ratio by Expiry')
axs[1].set_xlabel('Expiration Date')
axs[1].set_ylabel('Put/Call Ratio')
axs[1].grid(True)

# Third subplot: Implied Volatility Term Structure
term = []
for exp in ticker.options[:8]:
    oc = ticker.option_chain(exp)
    iv_mean = (oc.calls['impliedVolatility'].mean() + oc.puts['impliedVolatility'].mean()) / 2
    term.append({'expiry': exp, 'avg_IV': iv_mean})

df_term = pd.DataFrame(term)
axs[2].plot(df_term['expiry'], df_term['avg_IV'], marker='o', color='purple')
axs[2].set_title('Implied Volatility Term Structure')
axs[2].set_xlabel('Expiration Date')
axs[2].set_ylabel('Average Implied Volatility')
axs[2].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# --- Load data ---
expiry = ticker.options[0]  # nearest expiry
chain = ticker.option_chain(expiry)
calls, puts = chain.calls, chain.puts
spot = ticker.history(period="1d")['Close'].iloc[-1]

# --- Prepare data ---
strikes = np.array(sorted(set(calls['strike']).intersection(puts['strike'])))

# Select 10 strikes around the spot price
closest_idx = np.argmin(abs(strikes - spot))
start_idx = max(0, closest_idx - 4)
end_idx = min(len(strikes), closest_idx + 5)
strikes = strikes[start_idx:end_idx]

call_oi = calls.set_index('strike').reindex(strikes)['openInterest'].fillna(0)
put_oi  = puts.set_index('strike').reindex(strikes)['openInterest'].fillna(0)

# --- Plot settings ---
bar_width = 0.4
x = np.arange(len(strikes))

plt.figure(figsize=(12,6))
plt.bar(x - bar_width/2, call_oi, width=bar_width, color='green', alpha=0.7, label='Calls OI')
plt.bar(x + bar_width/2, put_oi,  width=bar_width, color='red', alpha=0.7, label='Puts OI')

# --- Add current stock price marker ---
closest_strike = strikes[np.argmin(abs(strikes - spot))]
plt.axvline(x[np.where(strikes == closest_strike)[0][0]], color='blue', linestyle='--', label=f'Spot Price (${spot:.2f})')

# --- Labels and style ---
plt.title(f"Open Interest by Strike — {expiry}")
plt.xlabel("Strike Price")
plt.ylabel("Open Interest")
plt.xticks(x, strikes, rotation=45)
plt.legend()
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# --- Load option chain ---
expiry = ticker.options[0]  # nearest expiry
chain = ticker.option_chain(expiry)
calls, puts = chain.calls, chain.puts

# --- Merge call and put open interest by strike ---
oi_table = pd.merge(
    calls[['strike', 'openInterest', 'volume', 'impliedVolatility']],
    puts[['strike', 'openInterest', 'volume', 'impliedVolatility']],
    on='strike',
    suffixes=('_call', '_put')
)

# --- Sort by strike ---
oi_table = oi_table.sort_values(by='strike').reset_index(drop=True)

# --- Format the columns for readability ---
oi_table.columns = [
    "Strike",
    "Call OI",
    "Call Volume",
    "Call IV",
    "Put OI",
    "Put Volume",
    "Put IV"
]

# --- Round and clean up ---
oi_table["Call IV"] = (oi_table["Call IV"] * 100).round(2)
oi_table["Put IV"] = (oi_table["Put IV"] * 100).round(2)

print(f"Open Interest Table — {expiry}")
display(oi_table.head(20))  # show first 20 rows